In [10]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from itertools import repeat


from snewpy import snowglobes
from astropy import units as u
from snewpy.neutrino import Flavor, MassHierarchy, MixingParameters
from snewpy.models.ccsn import *
from snewpy.flavor_transformation import AdiabaticMSW


SNOwGLoBES_path = "/Users/sierra/Desktop/neutrino_fall_2022/dependencies/snowglobes"
SNEWPY_models_base = "/Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models"  # directory containing SNEWPY models

# set parameters that don't vary over all models
# set distance in kpc
distance = 1.0

# set SNOwGLoBES detector to use
detector = "ar40kt"

# set desired flavor transformation
transformation = 'NoTransformation'

#window_tstart = 0.1 # Fornax
#window_tend = 1.0
#window_bins = 120

#window_tstart_burst = -0.1 # Fornax
#window_tend_burst = 0.1
#window_bins_burst = 60

#window_tstart = 0.1 # Nakazato
#window_tend = 3.6
#window_bins = 70

#window_tstart_burst = -0.1 # Nakazato
#window_tend_burst = 0.1
#window_bins_burst = 100


#window_tstart = 0.1 # Tamborra
#window_tend = 0.3
#window_bins = 4

#window_tstart_burst = -0.1 # Tamborra
#window_tend_burst = 0.1
#window_bins_burst = 100

window_tstart = 0.1015 # Fornax adjusted time bins
window_tend = 4.1015
window_bins = 40 # .1 s time bin

window_tstart_burst = -0.0485 # Fornax adjusted time bins
window_tend_burst = 0.1015
window_bins_burst = 15 # .01 s time bin



tstart = np.linspace(window_tstart, window_tend, window_bins, endpoint=False) 
tend = tstart + (window_tend - window_tstart) / window_bins 
tmid = (tstart + tend) * 0.5 

tstart_burst = np.linspace(window_tstart_burst, window_tend_burst, window_bins_burst, endpoint=False) 
tend_burst = tstart_burst + (window_tend_burst - window_tstart_burst) / window_bins_burst 
tmid_burst = (tstart_burst + tend_burst) * 0.5 

tstart = np.append(tstart_burst, tstart)
tend = np.append(tend_burst, tend)

# rename factor
#change_to_ms = window_bins / (window_tend - window_tstart) / 1000
#change_to_ms_burst = window_bins_burst / (window_tend_burst - window_tstart_burst) / 1000


# number of lines in dat file
num_lines = 500


mv Nakazato_2013_nakazato-shen-z0.004-t_rev100ms-s20.0.fits_AdiabaticMSW_NMO.tar.bz2 /Users/sierra/Desktop/neutrino/nakazato_files_fluence_phase/.



In [11]:
matrix = [['Fornax_2021', 'lum_spec_26.99M_r10000_dat.h5'],
         ['Nakazato_2013', 'nakazato-shen-z0.004-t_rev100ms-s20.0.fits'],
         ['Tamborra_2014', 's20.0c_3D_dir1'],
         ['Zha_2021', 's20.dat']]

modeltype = matrix[0][0]
model = matrix[0][1]

modelfile = SNEWPY_models_base + "/" + modeltype + "/" + model 
print(modelfile)
outfile = modeltype+"_"+model+"_"+transformation
print(outfile)
    # snowglobes.generate_fluence integrates the model over the specified time window(s)
    # and generates input files for SNOwGLoBES. It returns the full file path of the output file.
print("Preparing fluences ...")
tarredfile = snowglobes.generate_fluence(modelfile, modeltype, transformation, distance, outfile, tstart * u.s, tend * u.s)

    # Next, we run SNOwGLoBES. This will loop over all the fluence files in `tarredfile`.
print("Running SNOwGLoBES ...")
sim = snowglobes.simulate(SNOwGLoBES_path, tarredfile, detector_input=detector)


    # Finally, we collate SNOwGLoBES’ results into a dictionary
print("Collating results ...")
tables = snowglobes.collate(SNOwGLoBES_path, tarredfile, skip_plots=True)
print("Done")



print("mv " + SNEWPY_models_base + "/" + modeltype + "/" + outfile + "_SNOprocessed.tar.gz .")



print("mv " + SNEWPY_models_base + "/" + modeltype + "/" + outfile + ".tar.bz2 .") 
print("tar -xf " + outfile + ".tar.bz2")
print("rm parameterinfo")


/Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models/Fornax_2021/lum_spec_26.99M_r10000_dat.h5
Fornax_2021_lum_spec_26.99M_r10000_dat.h5_NoTransformation
Preparing fluences ...
Running SNOwGLoBES ...


Detectors:   0%|          | 0/1 [00:00<?, ?it/s]

Flux files:   0%|          | 0/55 [00:00<?, ?it/s]

Collating results ...
Done
mv /Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models/Fornax_2021/Fornax_2021_lum_spec_26.99M_r10000_dat.h5_NoTransformation_SNOprocessed.tar.gz .
mv /Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models/Fornax_2021/Fornax_2021_lum_spec_26.99M_r10000_dat.h5_NoTransformation.tar.bz2 .
tar -xf Fornax_2021_lum_spec_26.99M_r10000_dat.h5_NoTransformation.tar.bz2
rm parameterinfo


In [24]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from itertools import repeat


from snewpy import snowglobes
from astropy import units as u
from snewpy.neutrino import Flavor, MassHierarchy, MixingParameters
from snewpy.models.ccsn import *
from snewpy.flavor_transformation import AdiabaticMSW


SNOwGLoBES_path = "/Users/sierra/Desktop/neutrino_fall_2022/dependencies/snowglobes"
SNEWPY_models_base = "/Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models"  # directory containing SNEWPY models

# set parameters that don't vary over all models
# set distance in kpc
distance = 1.0

# set SNOwGLoBES detector to use
detector = "ar40kt"

# set desired flavor transformation
transformation = 'AdiabaticMSW_NMO'

#window_tstart = 0.1 # Fornax
#window_tend = 1.0
#window_bins = 120

#window_tstart_burst = -0.1 # Fornax
#window_tend_burst = 0.1
#window_bins_burst = 60

#window_tstart = 0.1 # Nakazato
#window_tend = 3.6
#window_bins = 70

#window_tstart_burst = -0.1 # Nakazato
#window_tend_burst = 0.1
#window_bins_burst = 100


#window_tstart = 0.1 # Tamborra
#window_tend = 0.3
#window_bins = 4

#window_tstart_burst = -0.1 # Tamborra
#window_tend_burst = 0.1
#window_bins_burst = 100

#/Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models/Bollig_2016/Bollig_2016_s27.0c_AdiabaticMSW_NMO
window_tstart = 0.1 # Zha
window_tend = 1.9
window_bins = 18

window_tstart_burst = -0.1 # Zha
window_tend_burst = 0.1
window_bins_burst = 20

# /Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models/Sukhbold_2015/Sukhbold_2015_sukhbold-LS220-s27.0.fits_AdiabaticMSW_NMO
# /Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models/Sukhbold_2015/Sukhbold_2015_sukhbold-SFHo-s27.0.fits_AdiabaticMSW_NMO
#window_tstart = 0.1 # Zha
#window_tend = 1.9
#window_bins = 360

#window_tstart_burst = -0.1 # Zha
#window_tend_burst = 0.1
#window_bins_burst = 100

# /Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models/Warren_2020/stir_a1.27/Warren_2020_stir_multimessenger_a1.27_m27.0.h5_AdiabaticMSW_NMO
# /Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models/Warren_2020/stir_a1.23/Warren_2020_stir_multimessenger_a1.23_m27.0.h5_AdiabaticMSW_NMO
"""
window_tstart = 0.1 # Zha
window_tend = 1.5
window_bins = 14

window_tstart_burst = -0.1 # Zha
window_tend_burst = 0.1
window_bins_burst = 20
"""
# /Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models/Sukhbold_2015/Sukhbold_2015_sukhbold-LS220-s27.0.fits_AdiabaticMSW_NMO
# /Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models/Sukhbold_2015/Sukhbold_2015_sukhbold-SFHo-s27.0.fits_AdiabaticMSW_NMO
#window_tstart = 0.1 # Zha
#window_tend = 1.9
#window_bins = 38

#window_tstart_burst = -0.1 # Zha
#window_tend_burst = 0.1
#window_bins_burst = 100


tstart = np.linspace(window_tstart, window_tend, window_bins, endpoint=False) 
tend = tstart + (window_tend - window_tstart) / window_bins 
tmid = (tstart + tend) * 0.5 

tstart_burst = np.linspace(window_tstart_burst, window_tend_burst, window_bins_burst, endpoint=False) 
tend_burst = tstart_burst + (window_tend_burst - window_tstart_burst) / window_bins_burst 
tmid_burst = (tstart_burst + tend_burst) * 0.5 

tstart = np.append(tstart_burst, tstart)
tend = np.append(tend_burst, tend)

# rename factor
change_to_s = window_bins / (window_tend - window_tstart) 
change_to_s_burst = window_bins_burst / (window_tend_burst - window_tstart_burst) 


# number of lines in dat file
num_lines = 500


In [26]:
matrix = [['Bollig_2016', 's27.0c'],
         ['Sukhbold_2015', 'sukhbold-LS220-s27.0.fits'],
         ['Sukhbold_2015', 'sukhbold-SFHo-s27.0.fits'],
         ['Warren_2020', 'stir_multimessenger_a1.27_m27.0.h5'],
         ['Warren_2020', 'stir_multimessenger_a1.23_m27.0.h5']]

modeltype = matrix[1][0]
model = matrix[1][1]

modelfile = SNEWPY_models_base + "/" + modeltype + "/" + model 
print(modelfile)
outfile = modeltype+"_"+model+"_"+transformation
print(outfile)
    # snowglobes.generate_fluence integrates the model over the specified time window(s)
    # and generates input files for SNOwGLoBES. It returns the full file path of the output file.
print("Preparing fluences ...")
tarredfile = snowglobes.generate_fluence(modelfile, modeltype, transformation, distance, outfile, tstart * u.s, tend * u.s)

    # Next, we run SNOwGLoBES. This will loop over all the fluence files in `tarredfile`.
#print("Running SNOwGLoBES ...")
sim = snowglobes.simulate(SNOwGLoBES_path, tarredfile, detector_input=detector)

    # Finally, we collate SNOwGLoBES’ results into a dictionary
print("Collating results ...")
tables = snowglobes.collate(SNOwGLoBES_path, tarredfile, skip_plots=True)
print("Done")

print("mv " + SNEWPY_models_base + "/" + modeltype + "/" + outfile + "_SNOprocessed.tar.gz . \n")



print("mv " + SNEWPY_models_base + "/" + modeltype + "/" + outfile + ".tar.bz2 .") 
print("tar -xf " + outfile + ".tar.bz2")
print("rm parameterinfo")



/Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models/Sukhbold_2015/sukhbold-LS220-s27.0.fits
Sukhbold_2015_sukhbold-LS220-s27.0.fits_AdiabaticMSW_NMO
Preparing fluences ...


Detectors:   0%|          | 0/1 [00:00<?, ?it/s]

Flux files:   0%|          | 0/38 [00:00<?, ?it/s]

Collating results ...
Done
mv /Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models/Sukhbold_2015/Sukhbold_2015_sukhbold-LS220-s27.0.fits_AdiabaticMSW_NMO_SNOprocessed.tar.gz . 

mv /Users/sierra/Desktop/neutrino_fall_2022/dependencies/SNEWPY_models/Sukhbold_2015/Sukhbold_2015_sukhbold-LS220-s27.0.fits_AdiabaticMSW_NMO.tar.bz2 .
tar -xf Sukhbold_2015_sukhbold-LS220-s27.0.fits_AdiabaticMSW_NMO.tar.bz2
rm parameterinfo


Collating results ...
Done
